In [6]:
dir_path = '/aiffel/aiffel/AIFFELTHON/data/test_1202/'
sub_dir = '/aiffel/aiffel/AIFFELTHON/submission/test_1202/'
file_name = 'train_en_fr_ch_12000'
bi_file_name = 'train_4000'

HUGGINGFACE_MODEL_PATH = "beomi/KcELECTRA-base-v2022"
HUGGINGFACE_MODEL_BINARY_PATH = "beomi/KcELECTRA-base-v2022"
HUGGINGFACE_MODEL_BERT_PATH = "jinmang2/klue-roberta-large-bt-tapt"

#model_name = 'KcELECTRA-2022'
model_name = 'KcELECTRA-2022-OOD-test'
#model_name = 'roberta-tapt'

batch_size = 8
num_epochs = 2

In [2]:
! pip install transformers
! pip install datasets

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tqdm
from transformers import Trainer, TrainingArguments
import re
import pyarrow as pa
from datasets import Dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from sklearn.model_selection import train_test_split
from datasets import load_metric

## 데이터 불러오기

In [4]:
data = pd.read_csv(dir_path+file_name+'.csv')

## 데이터 전처리

In [7]:
labels = {'협박 대화':0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3, '일반 대화':4}
binary_labels = {'협박 대화':0, '갈취 대화':0, '직장 내 괴롭힘 대화':0, '기타 괴롭힘 대화':0, '일반 대화':1}

num_labels = len(labels)
#binary_data = data.copy()
binary_data = pd.read_csv(dir_path+bi_file_name+'.csv')
attack_data = data.copy()

#attack_data = attack_data[attack_data['class'] != '일반 대화']

binary_data['label'] = binary_data['class'].apply(lambda x: binary_labels[x])
attack_data['label'] = attack_data['class'].apply(lambda x: labels[x])

## Tokenizer 및 Mdoel 불러오기

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification

binary_tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_BINARY_PATH)
binary_model = ElectraForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_BINARY_PATH, num_labels=2)

tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
model = ElectraForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=len(labels))

#tokenizer = BertTokenizer.from_pretrained(HUGGINGFACE_MODEL_BERT_PATH)
#model = BertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_BERT_PATH, num_labels=5)

Some weights of the model checkpoint at beomi/KcELECTRA-base-v2022 were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.weight', 

## 학습을 위한 데이터셋 생성

In [9]:
import pyarrow as pa
from datasets import Dataset

attack_data = Dataset(pa.Table.from_pandas(attack_data))
binary_data = Dataset(pa.Table.from_pandas(binary_data))

In [10]:
def transform(data):
  return tokenizer(
      data['non_label_sentence'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )
def bi_transform(data):
  return binary_tokenizer(
      data['non_label_sentence'],
      truncation = True,
      padding = 'max_length',
  )

encoded_data = attack_data.map(transform, batched=True)
encoded_binary_data = binary_data.map(bi_transform, batched=True)

  0%|          | 0/32 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

In [11]:
train_dataset, validation_dataset= encoded_data.train_test_split(test_size=0.1).values()
train_binary_dataset, validation_binary_dataset= encoded_binary_data.train_test_split(test_size=0.1).values()

In [12]:
metric = load_metric("f1")

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels, average="micro")

output_dir = '/content/drive/MyDrive/공부/AIFFELTHON/etc'

training_arguments = TrainingArguments(
    output_dir, 
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, 
    per_device_train_batch_size = batch_size, 
    per_device_eval_batch_size = batch_size, 
    num_train_epochs = num_epochs,
    weight_decay = 0.01
)

In [63]:
atk_trainer = Trainer(
    model=model,                           
    args=training_arguments,                  
    train_dataset=train_dataset,    
    eval_dataset=validation_dataset,      
    compute_metrics=compute_metrics
)

atk_trainer.train()

The following columns in the training set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: binary_class, label_sentence, type, non_label_sentence, class.
***** Running training *****
  Num examples = 28332
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7084


Epoch,Training Loss,Validation Loss,F1
1,0.015900,0.128256,0.985070
2,0.007900,0.107258,0.987611


Saving model checkpoint to /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-500
Configuration saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1000
Configuration saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1500
Configuration saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1500/config.json
Model weights saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-2000
Configuration saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/

TrainOutput(global_step=7084, training_loss=0.015622551438836712, metrics={'train_runtime': 5923.3933, 'train_samples_per_second': 9.566, 'train_steps_per_second': 1.196, 'total_flos': 1.4909326424580096e+16, 'train_loss': 0.015622551438836712, 'epoch': 2.0})

In [14]:
trainer = Trainer(
    model=binary_model,                           
    args=training_arguments,                  
    train_dataset=train_binary_dataset,    
    eval_dataset=validation_binary_dataset,      
    compute_metrics=compute_metrics
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: binary_class, label_sentence, type, non_label_sentence, class.
***** Running training *****
  Num examples = 7083
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1772


Epoch,Training Loss,Validation Loss,F1
1,0.101100,0.024721,0.992376
2,0.003200,0.034566,0.992376


Saving model checkpoint to /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-500
Configuration saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: binary_class, label_sentence, type, non_label_sentence, class.
***** Running Evaluation *****
  Num examples = 787
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1000
Configuration saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1500
Configuration saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/c

TrainOutput(global_step=1772, training_loss=0.043541253554901745, metrics={'train_runtime': 1475.884, 'train_samples_per_second': 9.598, 'train_steps_per_second': 1.201, 'total_flos': 3727231210229760.0, 'train_loss': 0.043541253554901745, 'epoch': 2.0})

In [13]:
# trainer = Trainer(
#     model=model,                           
#     args=training_arguments,                  
#     train_dataset=train_dataset,    
#     eval_dataset=validation_dataset,      
#     compute_metrics=compute_metrics
# )

# trainer.train()

In [15]:
test_file_path = '/aiffel/aiffel/AIFFELTHON/data/test.json'
with open(test_file_path, mode='rt', encoding='utf-8') as f:
    test_dataset = pd.read_json(f)

test_data = test_dataset.transpose()

In [16]:
def remove_punctuation(x):
  x = x.strip()
  x = re.sub("[^가-힣0-9]+", " ", x)
  x = re.sub("[ ]+", " ", x)
  x = x.strip()
  return x

test_data["text"] = test_data['text'].apply(lambda x : remove_punctuation(x))

In [17]:
test_X = Dataset(pa.Table.from_pandas(test_data))
#test_X = test_data['text']

In [18]:
def test_transform(data):
  return tokenizer(
      data['text'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )

encoded_test_data = test_X.map(test_transform, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
prediction = trainer.predict(encoded_test_data)

The following columns in the test set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: __index_level_0__, text.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


In [20]:
len(prediction.predictions)
pred_label = []

for i in range(len(prediction.predictions)):
    pred_label.append((str)(np.argmax(prediction.predictions[i])).zfill(2))
    
pred_label[:10]

['00', '00', '00', '01', '00', '00', '00', '00', '01', '00']

In [21]:
test_submission = test_data.assign(CLASS=pred_label)
test_submission = test_submission.rename(columns={'CLASS':'class'})

In [22]:
attack_test = test_submission[test_submission['class']=='00']

In [23]:
norm_submission = test_submission[test_submission['class'] == '01']
norm_submission['class'] = '04'

/tmp/ipykernel_167/3148698422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  norm_submission['class'] = '04'


In [24]:
attack_test.drop(['class'], axis=1, inplace=True)

/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [25]:
norm_submission

,text,class
t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,04
t_008,안경 안쓰고 키작고통통해보이는 분이었는데 다안경안씀 쌍꺼풀없었던것같고 아 단발인가 ...,04
t_011,아 취업하고 싶은데 누가 나 좀 안 데려가냐 아 그니까 나도 취업하고싶다 코로나때문...,04
t_016,응응 점점 깊이 깊이 이야기가 들어갈수록 응응 맞아 마음이 좀 그럴수 잇을것같은데 ...,04
t_027,와 강아지 너무 이쁘다 감사합니다 몇 살이에요 올해로 네 살이에요 와 애기네요 애기...,04
...,...,...
t_480,서울 관악아파트인데요 뒷 놀이터에서 한 학생이 맞고 있는거 같아요 정확한 주소좀 알...,04
t_484,오빠 나 카드값이 밀렸어 저번이 마지막이랬자나 이번에 내가 코로나로 일도 쉬구 힘들...,04
t_487,저녁 먹었어 아니 오늘 뭐먹을까 오늘 집에서 라면 먹자 무슨 라면 먹을건데 열라면에...,04
t_488,지인의 말만 듣고 가입한 보험이 살펴보니 제대로 보장해주는 게 하나도 없었어 지인의...,04


In [64]:
test_atk_X = Dataset(pa.Table.from_pandas(attack_test))
encoded_test_atk_data = test_atk_X.map(test_transform, batched=True)
atk_prediction = atk_trainer.predict(encoded_test_atk_data)

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: __index_level_0__, text.
***** Running Prediction *****
  Num examples = 429
  Batch size = 8


In [65]:
atk_pred_label = []

for i in range(len(atk_prediction.predictions)):
    atk_pred_label.append((str)(np.argmax(atk_prediction.predictions[i])).zfill(2))
    
atk_pred_label[:10]

['01', '02', '02', '03', '00', '00', '01', '01', '00', '02']

In [66]:
atk_submission = attack_test.assign(CLASS=atk_pred_label)
atk_submission = atk_submission.rename(columns={'CLASS':'class'})

In [67]:
submission = pd.concat([atk_submission, norm_submission])

In [68]:
submission.sort_index(inplace=True)

In [69]:
submission.drop(['text'], axis=1, inplace=True)
submission = submission.transpose()
submission

,t_000,t_001,t_002,t_003,t_004,t_005,t_006,t_007,t_008,t_009,...,t_490,t_491,t_492,t_493,t_494,t_495,t_496,t_497,t_498,t_499
class,01,02,02,04,03,00,00,01,04,01,...,03,04,00,01,01,02,01,01,02,00


In [70]:
import json

submission_file_path = sub_dir + 'submission_' + model_name + '10epochs' + file_name[5:] + '.json'
result = submission.to_json(submission_file_path)

with open(submission_file_path) as f:
    parsed = json.load(f)

with open(submission_file_path, 'w') as f:
    json.dump(parsed, f, indent=4)

In [34]:
# import json

# submission_file_path = sub_dir + 'submission_' + model_name + file_name[5:] + '.json'
# result = submission.to_json(submission_file_path)

# with open(submission_file_path) as f:
#     parsed = json.load(f)

# with open(submission_file_path, 'w') as f:
#     json.dump(parsed, f, indent=4)